# QCSchema

The [QCSchema](https://github.com/MolSSI/QCSchema) is a standard data format for quantum chemistry. The current version of it which has been adopted by multiple classical chemistry codes, only supports serialization via `JSON`, but their docs clearly indicate that `HDF5` may also be used. Thus, in Qiskit Nature, we have opted for implementing support for both hierarchical data formats.

Now, why do we have a tutorial specifically about this format you may wonder? The reason is fairly simple: this is the data format which our drivers use internally to transfer data between the classical computation and Qiskit Nature. Thus, this tutorial will explain a few concepts, in case you want to get a bit more elaborate with your driver interaction.

> **Note:** the support for electronic-repulsion integrals as part of the QCSchema is not yet part of the official specification and, thus, custom to Qiskit Nature. But we are working with the QCSchema authors to make this integration official!

For the purposes of this tutorial, we are using the `PySCFDriver`, but most discussion points should apply to the other electronic structure drivers, too.

First, let us construct a `PySCFDriver` and run it:

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver()

problem = driver.run()
print(problem)

This is basically short for the following:

In [2]:
from qiskit_nature.second_q.problems import ElectronicBasis

driver.run_pyscf()
problem = driver.to_problem(basis=ElectronicBasis.MO, include_dipole=True)
print(problem.basis)

ElectronicBasis.MO


There are two things to note here:
- the problem is specifically requested in the MO basis
- dipole integrals are handled separately (because the current QCSchema standard does not support these coefficients)

What this means for you as an end-user, is that you can also request the problem in another basis like so:

In [3]:
ao_problem = driver.to_problem(basis=ElectronicBasis.AO)
print(ao_problem.basis)

ElectronicBasis.AO


If you now want to transform an AO problem into the MO basis, you need to use the `BasisTransformer` which is explained in [a separate tutorial](05_problem_transformers.ipynb).

This is the point, where you need to understand that the `to_problem` method actually relies on the `to_qcschema` method internally:

In [4]:
from qiskit_nature.second_q.formats.qcschema_translator import qcschema_to_problem

qcschema = driver.to_qcschema()
ao_problem = qcschema_to_problem(qcschema, basis=ElectronicBasis.AO)

Specifically extracting the QCSchema object from the driver allows you to later extract a `BasisTransformer` from it, without having to manually dig out the AO-2-MO transformation coefficients from the depths of the driver object:

In [5]:
from qiskit_nature.second_q.formats.qcschema_translator import get_ao_to_mo_from_qcschema

basis_transformer = get_ao_to_mo_from_qcschema(qcschema)

In [6]:
mo_problem = basis_transformer.transform(ao_problem)
print(mo_problem.basis)

ElectronicBasis.MO


In [7]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright